![bluesteel](https://i1.wp.com/breakingintomodeling.com/wp-content/uploads/2017/04/Zoolander-Blue-Steel-look-min.png?resize=508%2C213&ssl=1 "modeling isn't easy...")

### Build a better model...

In [1]:
cd ..

/Users/johnphillips/Desktop/DSI-Class-Stuff/CAPSTONE


In [2]:
%matplotlib inline

In [23]:
%run __init__.py

In [10]:
# Start with reading the data from the pickle
teams = pd.read_pickle('data/teams.p')

In [11]:
# Separate the Numeric Columns:

teams_num = teams.select_dtypes(exclude=['object', 'category'])
teams_num.columns

Index([u'yearID', u'G', u'Ghome', u'W', u'L', u'LgWin', u'WSWin', u'R', u'AB',
       u'H', u'2B', u'3B', u'HR', u'BB', u'SO', u'SB', u'CS', u'HBP', u'SF',
       u'RA', u'ER', u'ERA', u'CG', u'SHO', u'SV', u'IPouts', u'HA', u'HRA',
       u'BBA', u'SOA', u'E', u'DP', u'FP', u'attendance', u'BPF', u'PPF',
       u'BA', u'OBP', u'SLG', u'OPS', u'WinP'],
      dtype='object')

### Separate X and y 

In [12]:
# Had to also drop Wins, Losses, and WinP since those are direct indicators.
# And they are not part of a players individual stats.
X = teams_num.drop(['WSWin', 'LgWin', 'W', 'L', 'WinP'], axis=1) 

# Make a y2 based upon Winning Leage Series, for possible second model.
y = teams_num['WSWin']
y2 = teams_num['LgWin'] 

In [13]:
# Train-test split our data...

# Standard split:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, test_size=0.5)

# Second split based upon LgWin target:
X2_train, X2_test, y2_train, y2_test = train_test_split(X, y2, random_state = 42, test_size=0.5)

### Improved Logistic Regression?

In [39]:
from sklearn.cross_validation import cross_val_score

logreg = LogisticRegression()
lr_scores = cross_val_score(logreg, X, y, cv=10, scoring='accuracy')
print(lr_scores.mean())

0.957328237525


### Better with KNN?

In [30]:
knn = KNeighborsClassifier(n_neighbors=5)
knn_scores = cross_val_score(knn, X, y, cv=10, scoring='accuracy')
print(knn_scores.mean())

0.9583820783


In [31]:
# Now Grid Search it:

k_range = list(range(1, 30))
param_grid = dict(n_neighbors=k_range)
grid = GridSearchCV(knn, param_grid, cv=10, scoring='accuracy')
grid.fit(X, y)

GridSearchCV(cv=10, error_score='raise',
       estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform'),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=0)

In [32]:
grid.grid_scores_

/Users/johnphillips/anaconda2/lib/python2.7/site-packages/sklearn/model_selection/_search.py:761: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


[mean: 0.92381, std: 0.01530, params: {'n_neighbors': 1},
 mean: 0.95732, std: 0.00244, params: {'n_neighbors': 2},
 mean: 0.95273, std: 0.00505, params: {'n_neighbors': 3},
 mean: 0.95838, std: 0.00202, params: {'n_neighbors': 4},
 mean: 0.95838, std: 0.00202, params: {'n_neighbors': 5},
 mean: 0.95873, std: 0.00151, params: {'n_neighbors': 6},
 mean: 0.95873, std: 0.00151, params: {'n_neighbors': 7},
 mean: 0.95873, std: 0.00151, params: {'n_neighbors': 8},
 mean: 0.95873, std: 0.00151, params: {'n_neighbors': 9},
 mean: 0.95873, std: 0.00151, params: {'n_neighbors': 10},
 mean: 0.95873, std: 0.00151, params: {'n_neighbors': 11},
 mean: 0.95873, std: 0.00151, params: {'n_neighbors': 12},
 mean: 0.95873, std: 0.00151, params: {'n_neighbors': 13},
 mean: 0.95873, std: 0.00151, params: {'n_neighbors': 14},
 mean: 0.95873, std: 0.00151, params: {'n_neighbors': 15},
 mean: 0.95873, std: 0.00151, params: {'n_neighbors': 16},
 mean: 0.95873, std: 0.00151, params: {'n_neighbors': 17},
 mean:

In [33]:
# What is the best model?

print(grid.best_score_)
print(grid.best_params_)
print(grid.best_estimator_)

0.95873015873
{'n_neighbors': 6}
KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=6, p=2,
           weights='uniform')


In [34]:
cross_val_score(grid, X, y, cv=10, scoring='roc_auc').mean()


0.52135835989041701